In [1]:
import os
import rasterio
import numpy as np
import pickle as pkl
from tqdm import tqdm
from rasterio.crs import CRS
from affine import Affine
from rasterio import windows
import math
import pandas as pd
from glob import glob
import pylas
from multiprocessing import Pool

In [2]:
def normalize_parameters(src):
        print("Performing channel normalization...")
        channel_normalisation = {}
        combined_channel = []
        assert(len(src.indexes) == 4) # Expects input tif to only have RGBI (4) channels
        for index in tqdm(src.indexes):
            # print(f"\r{index} of {len(src.indexes)}", end="")
            channel = src.read(index).flatten()
            combined_channel.extend(list(filter(lambda x: x > 0, channel))) # Remove black values
        
        combined_channel.sort()
        bottom_percentile = int(len(combined_channel)*.005)
        top_percentile = int(len(combined_channel)*.995)

        for index in src.indexes:
            channel_normalisation[index] = {
                "min": combined_channel[bottom_percentile], 
                "max": combined_channel[top_percentile]
            }
        print("\nDONE\n")
        return channel_normalisation

def normalise(tile, norm, w, h):
        newtile = []
        for i in tqdm(range(len(tile))):
            n = norm[i+1]
            d = n["max"]-n["min"]
            if i == 3:
                newtile.append(np.array([[255] * w]* h, dtype=np.uint8))
            newtile.append(
                np.array(
                    list(map( lambda x: 
                        list(map(lambda y: 
                            0 if y < 0 else 255 if y > 255 else y, x
                        )), 
                        [list(map(lambda x: int(round((float(x)-n["min"])/d*255)), row)) for row in tile[i]]
                    )
                ), dtype=np.uint8))
        return newtile
    
def generate_vegetation_height_channel(vegetation_height_data, vegetation_height_transform, target_transform, target_width, target_height):
        print("Generating vegetation height channel...")
        channel = np.array([[0] * target_width] * target_height, dtype=np.uint8)
        src_height, src_width = vegetation_height_data.shape[0], vegetation_height_data.shape[1]
        for y in tqdm(range(target_height)):
            for x in range(target_width):
                rd_x, rd_y = rasterio.transform.xy(target_transform, y, x)
                vh_y, vh_x = rasterio.transform.rowcol(vegetation_height_transform, rd_x, rd_y)
                if vh_x < 0 or vh_x >= src_width or \
                    vh_y < 0 or vh_y >= src_height:
                    continue
                # print(rd_x, rd_y, x, y, vh_x, vh_y)
                channel[y][x] = vegetation_height_data[vh_y][vh_x]
        return channel

def generate_ndvi_channel(tile):
        print("Generating NDVI channel...")
        red = tile[2]
        nir = tile[3]
        ndvi = []
        for i in tqdm(range(len(red))):
            ndvi.append((nir[i]-red[i])/(nir[i]+red[i]+1e-10)*255)
        return np.array(ndvi, dtype=np.uint8)

In [3]:
laz = pylas.read("C:/repos/satellite-images-nso/data/C_30EN2.LAZ")

In [4]:
ahn_output = "C:/repos/satellite-images-nso/data/output_ahn.tif"

In [5]:
date = laz.header.date    
cloud = np.dstack([laz.x, laz.y, laz.z, laz.classification])[0]

In [6]:
bb = rasterio.coords.BoundingBox(min(laz.x), min(laz.y),max(laz.x),max(laz.y))

In [7]:
cloud_df = pd.DataFrame({
            "x":laz.x, 
            "y":laz.y, 
            "z":laz.z,
            "classification": laz.classification
        })

In [8]:
del laz

In [ ]:
 for k in ["x", "y"]:
        cloud_df[k] = cloud_df[k].round()

In [ ]:
maaiveld_df = cloud_df[cloud_df["classification"] == 2]
vegetation_df = cloud_df[cloud_df["classification"] == 1]

In [ ]:
maaiveld_df = maaiveld_df.groupby(['x', 'y', "classification"]).agg({
                'z': 'min'
            })

maaiveld_df = maaiveld_df.reset_index()


vegetation_df = vegetation_df.groupby(['x', 'y', "classification"]).agg({
                'z': 'max'
            })

vegetation_df = vegetation_df.reset_index()

In [ ]:
ahn_data = pd.merge(vegetation_df, maaiveld_df, how='outer', on=['x', 'y'], suffixes=["_vegetation", "_maaiveld"])
ahn_data["height"] = list(map(lambda x: 0 if pd.isnull(x[0]) else x[0] - x[1], np.array([ahn_data["z_vegetation"], ahn_data["z_maaiveld"]]).T))

In [ ]:
del maaiveld_df
del vegetation_df

In [ ]:
ahn_data

,x,y,classification_vegetation,z_vegetation,classification_maaiveld,z_maaiveld,height
0,86661.0,468757.0,1.0,0.972,2.0,0.725,0.247
1,86661.0,468758.0,1.0,0.968,2.0,0.788,0.180
2,86662.0,468756.0,1.0,1.136,2.0,0.773,0.363
3,86662.0,468757.0,1.0,1.038,2.0,0.746,0.292
4,86662.0,468761.0,1.0,1.073,2.0,0.756,0.317
...,...,...,...,...,...,...,...
8070668,90000.0,474581.0,NaN,NaN,2.0,0.663,0.000
8070669,90000.0,474582.0,NaN,NaN,2.0,0.643,0.000
8070670,90000.0,474583.0,NaN,NaN,2.0,0.617,0.000
8070671,90000.0,474584.0,NaN,NaN,2.0,0.573,0.000


In [ ]:
def neighbor(x):
    nans[nans.index == x ]
    neighbors = [
                ahn_data[(ahn_data["x"]==row["x"]) & (ahn_data["y"]==row["y"]+1)],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"])],
                ahn_data[(ahn_data["x"]==row["x"]) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"])],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"]+1)],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"]+1)]
            ]
    
    heights = []
    for n in neighbors:
                if n.shape[0] == 0 or np.isnan(n["height"].values[0]): 
                    continue
                heights.append(n["height"].values[0])
    if len(heights) > minimalNeighborCount:
               ahn_data.at[idx, "height"] = np.median(heights)
    

In [ ]:
# fill the gaps
hasNans = True
minimalNeighborCount = 5
nanCount = 1e10

for k in tqdm(range(50)):
        nans = ahn_data[pd.isnull(ahn_data["height"])]
        hasNans = nans.shape[0] > 0
        if not hasNans:
            break
        if nanCount*.95 <= nans.shape[0] and minimalNeighborCount > 1:
            minimalNeighborCount -= 1
        nanCount = nans.shape[0]    
        print(nans.shape[0], minimalNeighborCount)

        for idx, row in nans.iterrows():
            neighbors = [
                ahn_data[(ahn_data["x"]==row["x"]) & (ahn_data["y"]==row["y"]+1)],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"])],
                ahn_data[(ahn_data["x"]==row["x"]) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"])],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"]+1)],
                ahn_data[(ahn_data["x"]==row["x"]+1) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"]-1)],
                ahn_data[(ahn_data["x"]==row["x"]-1) & (ahn_data["y"]==row["y"]+1)]
            ]
            heights = []
            for n in neighbors:
                if n.shape[0] == 0 or np.isnan(n["height"].values[0]): continue
                heights.append(n["height"].values[0])
            if len(heights) < minimalNeighborCount:
                continue
            ahn_data.at[idx, "height"] = np.median(heights)

min_x = ahn_data["x"].min()
min_y = ahn_data["y"].min()
max_x = ahn_data["x"].max()
max_y = ahn_data["y"].max()
res = 0.5
pixel_data = []

  0%|                                                                                        | 0/50 [00:00<?, ?it/s]

151231 5


  2%|█▍                                                                    | 1/50 [11:12:14<549:00:08, 40334.87s/it]

80240 5


  4%|██▊                                                                   | 2/50 [17:09:02<389:20:54, 29201.13s/it]

...

12875 3


 24%|█████████████████                                                      | 12/50 [43:46:51<61:32:09, 5829.72s/it]

12104 3


 26%|██████████████████▍                                                    | 13/50 [44:41:26<51:57:52, 5056.02s/it]

11616 2


In [ ]:
for x in range(int(((max_y - min_y)/res+2))):
        pixel_data.append([0] * int(((max_x - min_x)/res+2)))

for row in tqdm(ahn_data.iterrows()):
    if np.isnan(row[1]["height"]):
        continue

res = .5
x_index = int(round((row[1]["x"]-min_x)/res))
y_index = int(round((max_y - row[1]["y"])/res))
value = int(round((abs(row[1]["height"])*2550)**.5))
value = 0 if value < 0 else 254 if value > 254 else value
pixel_data[y_index][x_index] = value + 1
pixel_data[y_index+1][x_index] = value + 1
pixel_data[y_index][x_index+1] = value + 1
pixel_data[y_index+1][x_index+1] = value + 1

pixel_data = np.array([pixel_data], dtype=np.uint8)

print(pixel_data.shape)

meta = {
        'driver': 'GTiff', 
        'dtype': 'uint8', 
        'nodata': 0.0, 
        'width': pixel_data.shape[2], 
        'height': pixel_data.shape[1], 
        'count': 1, 
        'crs': CRS.from_epsg(28992), 
        'transform': Affine(.5, 0.0, min_x, 0.0, -.5, max_y)
}

with rasterio.open(ahn_output, 'w', **meta) as outds:        
    outds.write(pixel_data)

In [ ]:
def add_to_tif(tif_file,ahn_outpath):
    
    src = rasterio.open(tif_file)

    with rasterio.open(ahn_outpath, 'r') as inds:        
        vegetation_height_data = inds.read(1)
        vegetation_height_transform = inds.meta["transform"]
                        

    channel_normalisation = normalize_parameters(src)
    print("Channel remapping:",  channel_normalisation)


    
    

    with rasterio.open(os.path.join(in_path, input_filename)) as inds:
        outpath = normalized_input_filename    
        meta = inds.meta.copy()
        meta["count"] = 7
        meta["dtype"] = "uint8"
        win = windows.Window(col_off=0, row_off=0, width=meta["width"], height=meta["height"])        
        tile = inds.read(window=win) # TODO is this behaviour similar to inds.read() ? MW: yes
        #ndvi = generate_ndvi_channel(tile)
        normalized_tile = np.array(normalise(tile, channel_normalisation, meta["width"], meta["height"]))            
        heightChannel = generate_vegetation_height_channel(vegetation_height_data, vegetation_height_transform, inds.meta["transform"], meta["width"], meta["height"])
        normalized_tile = np.append(normalized_tile, [heightChannel], axis=0)
        #normalized_tile = np.append(normalized_tile, [ndvi], axis=0)
        with rasterio.open(outpath, 'w', **meta) as outds:        
            outds.write(normalized_tile)